<a href="https://colab.research.google.com/github/5502-DataMining/CovidSentiment/blob/main/Preprocessing%20%2B%20Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preprocessing and Modeling**

In [ ]:
from wordcloud import WordCloud
import re
import unicodedata
import nltk
import csv
import string
from textblob import TextBlob
from nltk.corpus import stopwords
import demoji

import transformers
from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

import pickle
import torch.nn as nn
import torch.nn.functional as F
SEED = 1024
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
from torch.utils.data import TensorDataset, DataLoader, Dataset, RandomSampler, SequentialSampler
import torchtext
from torchtext import data
import warnings
warnings.filterwarnings('ignore')
import operator
from sklearn.metrics import hamming_loss, jaccard_score, label_ranking_average_precision_score, f1_score
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

nltk.download('all')

In [51]:
!git clone https://github.com/5502-DataMining/CovidSentiment/

fatal: destination path 'CovidSentiment' already exists and is not an empty directory.


In [ ]:
demoji.download_codes()

In [ ]:
senwave = pd.read_csv("/content/COVID19_sentinentanalysissocialmedia/labeledEn.csv")
print("Length of Senwave Dataset = {}".format(len(senwave)))

In [ ]:
senwave.head(10)

# **Preprocessing**

In [ ]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [ ]:
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [ ]:
from COVID19_sentinentanalysissocialmedia.preprocessing.preprocess import preprocess
pp_class = preprocess(senwave, contractions, contractionsWithAnotherInvertedComma)
senwave['Tweet'] = senwave['Tweet'].apply(lambda x : pp_class.preprocess_tweet(x))

In [ ]:
senwave['Tweet'] = senwave['Tweet'].str.lower()
senwave.head(10)

In [ ]:
def check_coverage(vocab, embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:
            oov[word] = vocab[word]
            i += vocab[word]
            pass
        
    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key = operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose = True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except:
                vocab[word] = 1
    return vocab

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path, encoding = "utf-8") as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [ ]:
GLOVE_EMBEDDING_FILE = '/content/COVID19_sentinentanalysissocialmedia/glove.840B.300d.txt'
glove_embeddings = load_embeddings(GLOVE_EMBEDDING_FILE)
print(f'loaded {len(glove_embeddings)} word vectors ')

In [ ]:
vocab = build_vocab(list(senwave['Tweet'].apply(lambda x : x.split())))
oov = check_coverage(vocab, glove_embeddings)
oov[:10]

In [ ]:
from COVID19_sentinentanalysissocialmedia.preprocessing import wordReplace
senwave['Tweet'] = senwave['Tweet'].apply(lambda x : wordReplace.bruteGen(x))

# **Building the model followed by training & testing**

In [54]:
from sklearn.model_selection import train_test_split

sen_train, sen_test = train_test_split(senwave, test_size=0.2, random_state=1024)
sen_val, sen_test = train_test_split(sen_test, test_size=0.5, random_state=1024)

sen_train.to_csv("train.csv", index = False)
sen_test.to_csv("test.csv", index = False)
sen_val.to_csv("val.csv", index = False)

In [55]:
sen_train.shape, sen_val.shape, sen_test.shape

((8000, 13), (1000, 13), (1000, 13))

In [68]:
import spacy
spacy_en = spacy.load('en')

def tokenizer(tweet):
    tweet = re.sub(r'[\n]', ' ', tweet)
    return [tok.text for tok in spacy_en.tokenizer(tweet)]

TWEET = torchtext.legacy.data.Field(sequential = True, lower = True, tokenize = tokenizer)
LABEL = torchtext.legacy.data.Field(sequential = False, use_vocab = False)

dataFields = [("ID", None), ("Tweet", TWEET), ("Optimistic", LABEL), ("Thankful", LABEL),
              ("Empathetic", LABEL), ("Pessimistic", LABEL), ("Anxious", LABEL), ("Sad", LABEL),
              ("Annoyed", LABEL), ("Denial", LABEL), ("Official report", LABEL),
              ("Surprise", LABEL), ("Joking", LABEL)]

train_dataset, test_dataset, valid_dataset = torchtext.legacy.data.TabularDataset.splits(
    path = '/content/', train = 'train.csv', test = 'test.csv', validation = 'val.csv', format = 'csv', fields = dataFields, skip_header = True
    )

In [69]:
print("Number of training samples : {}\n Number of testing samples : {}\n Number of validation samples : {}".format(len(train_dataset), len(test_dataset), len(valid_dataset)))

Number of training samples : 8000
 Number of testing samples : 1000
 Number of validation samples : 1000


In [66]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TWEET.build_vocab(train_dataset, vectors = 'glove.840B.300d')

KeyboardInterrupt: ignored

#**Using BERT**

In [70]:
df = senwave.drop(['ID'], axis = 1)
df['list'] = df[df.columns[1:12]].values.tolist()
new_df = df[['Tweet', 'list']].copy()
new_df.head()

,Tweet,list
0,a glass of wine keeps the corona away drake ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,can anyone tell me if you took the flu shot la...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]"
2,by the way producers send me beats im working ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,when someone you know apart of your family d...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,dear soccer i really miss you please come ba...,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]"


**Preparing the dataset**

In [72]:
MAX_LEN = 200 #based on length of tweets
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
TEST_BATCH_SIZE = 1
EPOCHS = 4
LEARNING_RATE = 2e-5 #tried 1e-03, 1e-04, 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [95]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.tweet = dataframe['Tweet']
        self.targets = self.dataframe.list
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)
    
    def __getitem__(self, index):
        tweet = str(self.tweet[index])
        tweet = " ".join(tweet.split())
        target = self.targets[index]
        
        inputs = self.tokenizer.encode_plus(
            tweet,
            None,
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = self.max_len,
            return_token_type_ids = True,
            pad_to_max_length = True,
            return_attention_mask=True,
            return_tensors='pt',  # Return PyTorch tensors
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'tweet': tweet,
            'ids' : torch.tensor(ids, dtype = torch.long),
            'mask' : torch.tensor(mask, dtype = torch.long),
            'targets' : torch.tensor(target, dtype = torch.float)
        }

In [96]:
train_dataset = sen_train.drop(['ID'], axis = 1)
train_dataset['list'] = train_dataset[train_dataset.columns[1:12]].values.tolist()
train_df = train_dataset[['Tweet', 'list']].copy()
train_df = train_df.reset_index(drop = True)

test_dataset = sen_test.drop(['ID'], axis = 1)
test_dataset['list'] = test_dataset[test_dataset.columns[1:12]].values.tolist()
test_df = test_dataset[['Tweet', 'list']].copy()
test_df = test_df.reset_index(drop = True)

valid_dataset = sen_val.drop(['ID'], axis = 1)
valid_dataset['list'] = valid_dataset[valid_dataset.columns[1:12]].values.tolist()
valid_df = valid_dataset[['Tweet', 'list']].copy()
valid_df = valid_df.reset_index(drop = True)

In [97]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = CustomDataset(
      tokenizer = tokenizer,
      dataframe = df,
      max_len=max_len,

  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=0
  )

In [98]:
training_loader = create_data_loader(train_df, tokenizer, MAX_LEN, TRAIN_BATCH_SIZE)
validation_loader = create_data_loader(valid_df, tokenizer, MAX_LEN, VALID_BATCH_SIZE)
testing_loader = create_data_loader(test_df, tokenizer, MAX_LEN, TEST_BATCH_SIZE)

In [114]:
data = next(iter(training_loader))
data.keys()

print(data['ids'].squeeze(0).shape)
print(data['mask'].shape)
print(data['targets'].squeeze(0).shape)

torch.Size([1, 200])
torch.Size([1, 1, 200])
torch.Size([11])


**Creating the Transformer model**

In [ ]:
class BERT(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = nn.Dropout(0.3)
        self.layer3 = nn.Linear(768, 11)

    def forward(self, ids, mask):
        unw, out_1 = self.layer1(ids, attention_mask = mask)[0], self.layer1(ids, attention_mask = mask)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final

model = BERT()
model.to(device)

In [105]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = AdamW(params = model.parameters(), lr = LEARNING_RATE)
total_steps = len(training_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.BCEWithLogitsLoss().to(device)

**Fine Tuning**

In [118]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    ids = d["ids"].squeeze(0).to(device)
    mask = d["mask"].squeeze(0).to(device)
    targets = d["targets"].to(device)

    outputs = model(
      ids=ids,
      mask=mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [119]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      ids = d["ids"].squeeze(0).to(device)
      mask = d["mask"].squeeze(0).to(device)
      targets = d["targets"].to(device)

      outputs = model(
        ids=ids,
        mask=mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [120]:
%%time

EPOCHS = 4


history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    training_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(train_df)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    validation_loader,
    loss_fn, 
    device, 
    len(valid_df)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model, '/content/COVID19_sentinentanalysissocialmedia/models/bertmodel-new.pth')
    best_accuracy = val_acc
  print(best_accuracy)

Epoch 1/4
----------
Train loss 0.36887306316103785 accuracy 1.0005
Val   loss 0.33389336156472565 accuracy 0.89

tensor(0.8900, device='cuda:0', dtype=torch.float64)
Epoch 2/4
----------
Train loss 0.28592678264610005 accuracy 1.11775
Val   loss 0.36135224744305017 accuracy 1.226

tensor(1.2260, device='cuda:0', dtype=torch.float64)
Epoch 3/4
----------
Train loss 0.2147768516429933 accuracy 1.1195
Val   loss 0.4114081586119719 accuracy 1.348

tensor(1.3480, device='cuda:0', dtype=torch.float64)
Epoch 4/4
----------
Train loss 0.15852845168864588 accuracy 1.1027500000000001
Val   loss 0.4343771455541719 accuracy 1.199

tensor(1.3480, device='cuda:0', dtype=torch.float64)
CPU times: user 2h 4min 20s, sys: 2min, total: 2h 6min 20s
Wall time: 2h 6min 16s


In [1]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

NameError: ignored

In [ ]:
# def train(epoch):
#     model.train()
#     total_loss = 0

#     for unw, data in enumerate(training_loader, 0):
#         ids = data['ids'].to(device, dtype = torch.long)
#         mask = data['mask'].to(device, dtype = torch.long)
#         token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#         targets = data['targets'].to(device, dtype = torch.float)

#         outputs = model(ids, mask, token_type_ids, return_dict = False)
#         loss = loss_fn(outputs, targets)
#         total_loss += loss.item()

#         if unw % 2000 == 0:
#             print(f'Iter : {unw+1}, Epoch: {epoch+1}, Loss: {total_loss/(unw+1)}')

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

In [ ]:
# EPOCHS = 4
# for epoch in range(EPOCHS):
#     train(epoch)

**Validating the data**

In [ ]:
# def valid():
#     model.eval()
#     req_targets = []
#     req_outputs = []
#     valid_loss = 0

#     with torch.no_grad():
#         for unw, data in enumerate(testing_loader, 0):
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)
#             outputs = model(ids, mask, token_type_ids)
#             loss = loss_fn(outputs, targets)
#             valid_loss += loss.item()

#             req_targets.extend(targets.cpu().detach().numpy().tolist())
#             req_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
#     valid_loss /= len(testing_loader)
#     return req_outputs, req_targets, valid_loss

In [ ]:
# from sklearn import metrics

# outputs, targets, valid_loss = valid()

In [ ]:
# outputs = np.array(outputs)
# targets = np.array(targets)

In [ ]:
# int_outputs = np.zeros((outputs.shape[0], outputs.shape[1]))

# for row in range(outputs.shape[0]):
#     for col in range(outputs.shape[1]):
#         if outputs[row][col] >= 0.5: int_outputs[row][col] = 1

In [ ]:
# targets[0], int_outputs[0], outputs[0]

In [ ]:
# bert_ham_loss = hamming_loss(targets, int_outputs)
# bert_jacc_score = jaccard_score(targets, int_outputs, average = 'samples')
# bert_lrap = label_ranking_average_precision_score(targets, outputs)
# bert_f1_macro = f1_score(targets, int_outputs, average = 'macro')
# bert_f1_micro = f1_score(targets, int_outputs, average = 'micro')

In [ ]:
# print("Test Loss:", valid_loss)
# print("Hamming Loss:", bert_ham_loss)
# print("Jaccard Score:", bert_jacc_score)
# print("Label Ranking Average Precision Score:", bert_lrap)
# print("F1 Macro Score:", bert_f1_macro)
# print("F1 Micro Score:", bert_f1_micro)

Results:


1.   lr = 1e-03: bce loss: 0.432, hamming loss: 0.183, jacc score: 0, lrap: 0.5659, f1 macro: 0, f1 micro: 0
2.   lr = 1e-04: bce loss: 0.432, hamming loss: 0.183, jacc score: 0, lrap: 0.5275, f1 macro: 0, f1 micro: 0
3.   lr = 1e-05: bce loss: 0.372, hamming loss: 0.142, jacc score: 0.5096, lrap: 0.766, f1 macro: 0.53, f1 micro: 0.587 



In [ ]:
# torch.save(model, f = '/content/COVID19_sentinentanalysissocialmedia/models/bertmodel.pth')

# **Using LSTM**

**Hyperparameters to be considered**
*  Learning Rate
*  Hidden Dimension of LSTM
*  Dropout Probability
*  Threshold



In [ ]:
vocab = TWEET.vocab
BATCH_SIZE = 32
n_label = 11
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, test_iter = data.BucketIterator.splits(datasets = (train_dataset, test_dataset),
                                                   batch_size = BATCH_SIZE,
                                                   sort_key = lambda x : len(x.Tweet),
                                                   sort_within_batch = False,
                                                   repeat = False,
                                                   device = device)

In [ ]:
np.random.seed(1024)
for i in range(TWEET.vocab.vectors.shape[0]):
    vec = TWEET.vocab.vectors[i]
    if torch.sum(vec).item() == 0:
        a = np.random.uniform(-0.25, 0.25, 300)
        TWEET.vocab.vectors[i] = torch.from_numpy(a)

In [ ]:
batch = next(train_iter.__iter__()); batch

In [ ]:
#idea taken from http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
class BatchWrapper():
    def __init__(self, dl, x_var, y_vars):
        self.dl = dl
        self.x_var = x_var
        self.y_vars = y_vars

    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var)
            if self.y_vars is not None:
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim = 1).float()
            else:
                y = torch.zeros((1))
            yield(x, y)
    
    def __len__(self):
        return len(self.dl)

In [ ]:
train_dl = BatchWrapper(train_iter, "Tweet", ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 
                                              'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'])
test_dl = BatchWrapper(test_iter, "Tweet", ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 
                                            'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'])

In [ ]:
class CustomLSTM(nn.Module):
    def __init__(self, embedding_dim, vocab, hidden_dim, output_dim, drop_prob, bidirectional = False, use_glove = True):
        super().__init__()
        self.embeddings = nn.Embedding(len(vocab), embedding_dim)
        if use_glove:
            self.embeddings.weight.data.copy_(vocab.vectors)
            self.embeddings.weight.requires_grad = False
        self.drop_prob = drop_prob

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional = bidirectional, batch_first = True, num_layers = 2)
        if bidirectional is True:
            self.lin = nn.Linear(2*hidden_dim, 64)
        else:
            self.lin = nn.Linear(hidden_dim, 64)
        self.fc = nn.Linear(64, output_dim)
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, sentence):
        #sentence = [max_len, batch_size]

        embed = self.embeddings(torch.transpose(sentence, 0, 1))
        #embed = [batch_size, max_len, embedding_dim]
        
        if self.drop_prob:
            embed = self.dropout(embed)
        
        lstm_out, (hidden, cell) = self.lstm(embed)
        #lstm_out = [batch_size, max_len, 2*hidden_dim if bidirectional else hidden_dim]
        #hidden = [num_layers, batch_size, hidden_dim]
        #cell = [num_layers, batch_size, hidden_dim]
        
        out = lstm_out[:,-1,:].squeeze()
        #out = [batch_size, 2*hidden_dim if bidirectional else hidden_dim]
        
        out = self.lin(out)
        #out = [batch_size, 64]

        outputs = self.fc(out)
        #outputs = [batch_size, output_dim]
        
        return outputs

In [ ]:
def evaluation_metrics(actual_labels, pred_labels, threshold):
    int_pred_labels = pred_labels
    for i in range(len(pred_labels)):
        for j in range(11):
            if int_pred_labels[i][j] >= threshold: int_pred_labels[i][j] = 1
            else:
                int_pred_labels[i][j] = 0
    
    ham_loss = hamming_loss(actual_labels, int_pred_labels)
    jacc_score = jaccard_score(actual_labels, int_pred_labels, average = 'samples')
    lrap = label_ranking_average_precision_score(actual_labels, pred_labels)
    f1_macro = f1_score(actual_labels, int_pred_labels, average = 'macro')
    f1_micro = f1_score(actual_labels, int_pred_labels, average = 'micro')

    return ham_loss, jacc_score, lrap, f1_macro, f1_micro

In [ ]:
def train(model, loss_fn, optimizer, n_epochs, train_dl, threshold):

    train_losses = []  
    hamming_losses = []
    jaccard_scores = []
    lraps = []  
    iter = 1

    for epoch in range(1, n_epochs+1):
        running_loss = 0.0
        pred_labels = []
        actual_labels = []
        model.train()
        for x, y in train_dl:
            #print(x.shape, y.shape)
            optimizer.zero_grad()

            preds = model(x)

            m = nn.Sigmoid()
            sig_preds = m(preds)
            
            for tens in sig_preds:
                pred_labels.append(tens.cpu().detach().numpy())
            for tens in y:
                actual_labels.append(tens.cpu().detach().numpy())

            loss = loss_fn(preds, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.shape[0]

        ham_loss, jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

        epoch_loss = running_loss / len(train_dataset)
        train_losses.append(epoch_loss)
        hamming_losses.append(ham_loss)
        lraps.append(lrap)
        jaccard_scores.append(jacc_score)
        '''
        if iter % 5 == 0:
            print("Epoch: ", epoch)
            print("Binary Cross Entropy With Logits Loss: {:.4f}".format(epoch_loss))
            print("Hamming Loss : {:.4f}".format(ham_loss))
            print("Jaccard Score: {:.4f}".format(jacc_score))
            print("Label Ranking Average Precision Score: {:.4f}".format(lrap))
            print("F1 Macro Score: {:.4f}".format(f1_macro))
            print("F1 Micro Score: {:.4f}".format(f1_micro))
            print("\n")
        iter += 1
        '''
    return train_losses, hamming_losses, jaccard_scores, lraps, f1_macro, f1_micro

In [ ]:
def test(model, loss_fn, test_dl, threshold):
    running_loss = 0.0
    pred_labels = []
    actual_labels = []
    model.eval()
    for x, y in test_dl:
        #print(x.shape, y.shape)

        preds = model(x)

        m = nn.Sigmoid()
        sig_preds = m(preds)
        
        for tens in sig_preds:
            pred_labels.append(tens.cpu().detach().numpy())
        for tens in y:
            actual_labels.append(tens.cpu().detach().numpy())

        loss = loss_fn(preds, y)

        running_loss += loss.item() * x.shape[0]

    ham_loss, jacc_score, lrap, f1_macro, f1_micro = evaluation_metrics(actual_labels, pred_labels, threshold)

    test_loss = running_loss / len(test_dataset)
    return test_loss, ham_loss, jacc_score, lrap, f1_macro, f1_micro

# **Experimentation with various combinations of the hyperparameters**

## Till 22-12-2020

In [ ]:
learning_rates = [1e-3]
hidden_dims = [128, 256]
thresholds = [0.4, 0.45, 0.5]
dropouts = [True, False]

all_models = []
iter = 1
#number of iterations = 3*2*2 = 12 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for dropout in dropouts:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = dropout)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
                all_models.append(model)
                print("Iteration : {}".format(iter))
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout = {}".format(learning_rate, hidden_dim, threshold, dropout))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

## 23-12-2020
*  taking inference from the previous days, having a dropout in the LSTM layer has always lowered the overfitting. Hence we will be using dropout layers in all models today.
*  We shall experiment with the values of the dropout probability today

In [ ]:
learning_rates = [1e-3]
hidden_dims = [128, 256]
thresholds = [0.4, 0.45, 0.5]
drop_probs = [0.4, 0.5, 0.6, 0.7]

all_models = []
iter = 1
#number of iterations = 4*3*2 = 24 = number of models

for learning_rate in learning_rates:
    for hidden_dim in hidden_dims:
        for threshold in thresholds:
            for drop_prob in drop_probs:
                model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = drop_prob)
                model = model.to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
                loss_fn = nn.BCEWithLogitsLoss()

                train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
                all_models.append(model)
                print("Iteration : {}".format(iter))
                print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout Probability = {}".format(learning_rate, hidden_dim, threshold, drop_prob))
                print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
                print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
                print("Jaccard Score: {:.4f}".format(max(jacc_score)))
                print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
                print("F1 Macro Score: {:.4f}".format(f1_macro))
                print("F1 Micro Score: {:.4f}".format(f1_micro))
                print("\n")

                test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
                print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
                print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
                print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
                print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
                print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
                print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
                print("\n")
                print("----------------------------------------------------------------")
                iter += 1

# **Some inferences**
*  Best Test BCE with logits loss at Iteration 12<br>
*  Best Test Hamming Loss at Iteration 12<br>
*  Best Test Jaccard Score at Iteration 3 & 16<br>
*  Best Test LRAP Score at Iteration 11<br>
*  Best Test F1 Macro Score at Iteration 16<br>
*  Best Test F1 Micro Score at Iteration 3<br>
<br> <br>
*  This indicates that we need only consider hyperparams corresponding to iterations 3, 11, 12 and 16



 **Hyperparameters to be considered for the final steps**:

*   Learning Rate: 0.001
*   Hidden Dimension: 3, 11, 12 = 128 & 16 = 256
*   Threshold: 3 & 16 = 0.4, 11 & 12 = 0.5
*   Dropout Probability: 3 & 11 = 0.6, 12 & 16 = 0.7

Since we see that either the best hyperparameters correspond to dropout probability = 0.6 or 0.7, we use dropout probability = 0.65 which is verified below as performing nearly the same or better

## Final evaluation using narrowed down hyperparameters
26-12-2020

In [ ]:
%%time
learning_rate = 1e-3
hidden_dim = 128
threshold = 0.5
drop_prob = 0.65

all_bce_losses = []
all_hamm_losses = []
all_jacc_scores = []
all_lraps = []
all_f1_macro = []
all_f1_micro = []
all_models = []

for exp in range(1, 11):

    print("Experiment {}".format(exp), '\n\n')

    model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = drop_prob)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    loss_fn = nn.BCEWithLogitsLoss()

    train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
    torch.save(model, 'model{}.pth'.format(exp))
    '''
    print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout Probability = {}".format(learning_rate, hidden_dim, threshold, drop_prob))
    print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
    print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
    print("Jaccard Score: {:.4f}".format(max(jacc_score)))
    print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
    print("F1 Macro Score: {:.4f}".format(f1_macro))
    print("F1 Micro Score: {:.4f}".format(f1_micro))
    print("\n")
    '''

    test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
    all_bce_losses.append(test_loss)
    all_hamm_losses.append(test_hamm_loss)
    all_jacc_scores.append(test_jacc_score)
    all_lraps.append(test_lrap)
    all_f1_macro.append(test_f1_macro)
    all_f1_micro.append(test_f1_micro)


    print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
    print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
    print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
    print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
    print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
    print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
    print("\n")

In [ ]:
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(all_bce_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(all_hamm_losses)/10))
print("Average Jaccard Score: {:.4f}".format(sum(all_jacc_scores)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(all_lraps)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(all_f1_macro)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(all_f1_micro)/10))

**Model 7 performed the best, so I downloaded the weights**

## Using a different value of dropout probability

In [ ]:
%%time
learning_rate = 1e-3
hidden_dim = 128
threshold = 0.5
drop_prob = 0.6

all_bce_losses = []
all_hamm_losses = []
all_jacc_scores = []
all_lraps = []
all_f1_macro = []
all_f1_micro = []
all_models = []

for exp in range(1, 11):

    print("Experiment {}".format(exp), '\n\n')

    model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = drop_prob)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    loss_fn = nn.BCEWithLogitsLoss()

    train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
    torch.save(model, 'model{}.pth'.format(exp))
    '''
    print("Learning Rate : {}, Hidden Dimension : {}, Threshold : {}, Dropout Probability = {}".format(learning_rate, hidden_dim, threshold, drop_prob))
    print("Binary Cross Entropy With Logits Loss: {:.4f}".format(min(train_loss)))
    print("Hamming Loss : {:.4f}".format(min(hamm_loss)))
    print("Jaccard Score: {:.4f}".format(max(jacc_score)))
    print("Label Ranking Average Precision Score: {:.4f}".format(max(lrap)))
    print("F1 Macro Score: {:.4f}".format(f1_macro))
    print("F1 Micro Score: {:.4f}".format(f1_micro))
    print("\n")
    '''

    test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
    all_bce_losses.append(test_loss)
    all_hamm_losses.append(test_hamm_loss)
    all_jacc_scores.append(test_jacc_score)
    all_lraps.append(test_lrap)
    all_f1_macro.append(test_f1_macro)
    all_f1_micro.append(test_f1_micro)


    print("TestBinary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
    print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
    print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
    print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
    print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
    print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
    print("\n")

In [ ]:
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(all_bce_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(all_hamm_losses)/10))
print("Average Jaccard Score: {:.4f}".format(sum(all_jacc_scores)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(all_lraps)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(all_f1_macro)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(all_f1_micro)/10))

## Comparison between having GloVe Vectors and not having GloVe Vectors 
28-12-2020

In [ ]:
%%time
learning_rate = 1e-3
hidden_dim = 128
threshold = 0.5
drop_prob = 0.65

all_glove_losses = []
all_glove_ham = []
all_glove_jacc = []
all_glove_lrap = []
all_glove_f1mac = []
all_glove_f1mic = []

all_nonglove_losses = []
all_nonglove_ham = []
all_nonglove_jacc = []
all_nonglove_lrap = []
all_nonglove_f1mac = []
all_nonglove_f1mic = []

glove = [True, False]
for exp in range(1, 11):
    for g in glove:
        model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = drop_prob, use_glove = g)
        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
        loss_fn = nn.BCEWithLogitsLoss()

        train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
        test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
        print("Test Binary Cross Entropy With Logits Loss: {:.4f}".format(test_loss))
        print("Test Hamming Loss : {:.4f}".format(test_hamm_loss))
        print("Test Jaccard Score: {:.4f}".format(test_jacc_score))
        print("Test Label Ranking Average Precision Score: {:.4f}".format(test_lrap))
        print("Test F1 Macro Score: {:.4f}".format(test_f1_macro))
        print("Test F1 Micro Score: {:.4f}".format(test_f1_micro))
        print("\n")

        if g is True:
            all_glove_losses.append(test_loss)
            all_glove_ham.append(test_hamm_loss)
            all_glove_jacc.append(test_jacc_score)
            all_glove_lrap.append(test_lrap)
            all_glove_f1mac.append(test_f1_macro)
            all_glove_f1mic.append(test_f1_micro)
        else:
            all_nonglove_losses.append(test_loss)
            all_nonglove_ham.append(test_hamm_loss)
            all_nonglove_jacc.append(test_jacc_score)
            all_nonglove_lrap.append(test_lrap)
            all_nonglove_f1mac.append(test_f1_macro)
            all_nonglove_f1mic.append(test_f1_micro)

In [ ]:
print("Using glove vectors:")
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(all_glove_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(all_glove_ham)/10))
print("Average Jaccard Score: {:.4f}".format(sum(all_glove_jacc)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(all_glove_lrap)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(all_glove_f1mac)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(all_glove_f1mic)/10))
print("\n\n")

print("Not using glove vectors:")
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(all_nonglove_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(all_nonglove_ham)/10))
print("Average Jaccard Score: {:.4f}".format(sum(all_nonglove_jacc)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(all_nonglove_lrap)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(all_nonglove_f1mac)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(all_nonglove_f1mic)/10))

In [ ]:
print(np.std(np.array(all_glove_losses)), np.std(np.array(all_glove_ham)), np.std(np.array(all_glove_jacc)), np.std(np.array(all_glove_lrap)),
      np.std(np.array(all_glove_f1mac)), np.std(np.array(all_glove_f1mic)))

print(np.std(np.array(all_nonglove_losses)), np.std(np.array(all_nonglove_ham)), np.std(np.array(all_nonglove_jacc)), np.std(np.array(all_nonglove_lrap)),
      np.std(np.array(all_nonglove_f1mac)), np.std(np.array(all_nonglove_f1mic)))

Here we see that having pretrained word vectors helps

## Comparing a unidirectional and a bidirectional network
03-01-2021

update - 10-01-2021

In [ ]:
learning_rate = 1e-3
hidden_dim = 128
threshold = 0.5
drop_prob = 0.65

iter = 1
bce_losses = []
ham_losses = []
jacc_scores = []
lrap_scores = []
f1_macros = []
f1_micros = []

bi_bce_losses = []
bi_ham_losses = []
bi_jacc_scores = []
bi_lrap_scores = []
bi_f1_macros = []
bi_f1_micros = []

for exp in range(1, 11):
    for dirn in [True, False]:
        model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, 
                            drop_prob = drop_prob, bidirectional = dirn, use_glove = True)
        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
        loss_fn = nn.BCEWithLogitsLoss()

        train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
        test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
        
        if dirn is True:
            bi_bce_losses.append(test_loss)
            bi_ham_losses.append(test_hamm_loss)
            bi_jacc_scores.append(test_jacc_score)
            bi_lrap_scores.append(test_lrap)
            bi_f1_macros.append(test_f1_macro)
            bi_f1_micros.append(test_f1_micro)
        else:
            bce_losses.append(test_loss)
            ham_losses.append(test_hamm_loss)
            jacc_scores.append(test_jacc_score)
            lrap_scores.append(test_lrap)
            f1_macros.append(test_f1_macro)
            f1_micros.append(test_f1_micro)

        print("Iteration {} / {} done".format(iter, 20))
        iter += 1

In [ ]:
print("Using bidirectional lstm:")
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(bi_bce_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(bi_ham_losses)/10))
print("Average Jaccard Score: {:.4f}".format(sum(bi_jacc_scores)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(bi_lrap_scores)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(bi_f1_macros)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(bi_f1_micros)/10))
print("\n\n")

print("Using lstm:")
print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(bce_losses)/10))
print("Average Hamming Loss : {:.4f}".format(sum(ham_losses)/10))
print("Average Jaccard Score: {:.4f}".format(sum(jacc_scores)/10))
print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(lrap_scores)/10))
print("Average F1 Macro Score: {:.4f}".format(sum(f1_macros)/10))
print("Average F1 Micro Score: {:.4f}".format(sum(f1_micros)/10))

In [ ]:
print("Using bidirectional lstm:")
print("Deviation of Binary Cross Entropy With Logits Loss: {:.4f}".format(np.std(np.array(bi_bce_losses))))
print("Deviation of Hamming Loss : {:.4f}".format(np.std(np.array(bi_ham_losses))))
print("Deviation of Jaccard Score: {:.4f}".format(np.std(np.array(bi_jacc_scores))))
print("Deviation of Label Ranking Average Precision Score: {:.4f}".format(np.std(np.array(bi_lrap_scores))))
print("Deviation of F1 Macro Score: {:.4f}".format(np.std(np.array(bi_f1_macros))))
print("Deviation of F1 Micro Score: {:.4f}".format(np.std(np.array(bi_f1_micros))))
print("\n\n")

print("Using lstm:")
print("Deviation of Binary Cross Entropy With Logits Loss: {:.4f}".format(np.std(np.array(bce_losses))))
print("Deviation of Hamming Loss : {:.4f}".format(np.std(np.array(ham_losses))))
print("Deviation of Jaccard Score: {:.4f}".format(np.std(np.array(jacc_scores))))
print("Deviation of Label Ranking Average Precision Score: {:.4f}".format(np.std(np.array(lrap_scores))))
print("Deviation of F1 Macro Score: {:.4f}".format(np.std(np.array(f1_macros))))
print("Deviation of F1 Micro Score: {:.4f}".format(np.std(np.array(f1_micros))))

we see that a bidirectional network performs much worse than a unidirectional network with the same set of parameters

## Comparing dropout values of 0, 0.5, 0.6, 0.7 for adding in the table
06-01-2021

update - 10-01-2021

In [ ]:
learning_rates = [1e-3]
hidden_dims = [128]
drop_probs = [0, 0.5, 0.6, 0.7]

for drop_prob in drop_probs:
    bce_losses = []
    ham_losses = []
    jacc_scores = []
    lrap_scores = []
    f1_macros = []
    f1_micros = []
    for exp in range(1, 11):
        model = CustomLSTM(embedding_dim = vocab.vectors.shape[1], vocab = vocab, hidden_dim = hidden_dim, output_dim = 11, drop_prob = drop_prob)
        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
        loss_fn = nn.BCEWithLogitsLoss()

        train_loss, hamm_loss, jacc_score, lrap, f1_macro, f1_micro = train(model, loss_fn, optimizer, 50, train_dl, threshold)
        #all_models.append(model)

        test_loss, test_hamm_loss, test_jacc_score, test_lrap, test_f1_macro, test_f1_micro = test(model, loss_fn, test_dl, threshold)
        bce_losses.append(test_loss)
        ham_losses.append(test_hamm_loss)
        jacc_scores.append(test_jacc_score)
        lrap_scores.append(test_lrap)
        f1_macros.append(test_f1_macro)
        f1_micros.append(test_f1_micro)
    
    print("Dropout = {}".format(drop_prob))
    print("Average Binary Cross Entropy With Logits Loss: {:.4f}".format(sum(bce_losses)/10))
    print("Average Hamming Loss : {:.4f}".format(sum(ham_losses)/10))
    print("Average Jaccard Score: {:.4f}".format(sum(jacc_scores)/10))
    print("Average Label Ranking Average Precision Score: {:.4f}".format(sum(lrap_scores)/10))
    print("Average F1 Macro Score: {:.4f}".format(sum(f1_macros)/10))
    print("Average F1 Micro Score: {:.4f}".format(sum(f1_micros)/10))
    print("\n\n")

    print("Deviation of Binary Cross Entropy With Logits Loss: {:.4f}".format(np.std(np.array(bce_losses))))
    print("Deviation of Hamming Loss : {:.4f}".format(np.std(np.array(ham_losses))))
    print("Deviation of Jaccard Score: {:.4f}".format(np.std(np.array(jacc_scores))))
    print("Deviation of Label Ranking Average Precision Score: {:.4f}".format(np.std(np.array(lrap_scores))))
    print("Deviation of F1 Macro Score: {:.4f}".format(np.std(np.array(f1_macros))))
    print("Deviation of F1 Micro Score: {:.4f}".format(np.std(np.array(f1_micros))))
    print("\n\n")

In [ ]:
print("BCE Losses: ", bce_losses)
print("Hamming Losses: ", ham_losses)
print("Jaccard Scores: ", jacc_scores)
print("LRAP Scores: ", lrap_scores)
print("F1 Macros: ", f1_macros)
print("F1 Micros: ", f1_micros)

## Modeling on IEEE Dataset

In [ ]:
ieee = pd.read_csv("/content/drive/MyDrive/Covid 19 India/ieee_extract.csv")
ieee['tweet'] = ieee['tweet'].str.lower()
ieee.head(10)

In [ ]:
def emoji2text(tweet):
    emojis = demoji.findall(tweet)
    new_tweet = []
    for word in tweet.split():
        if word in emojis.keys():
            tweet = tweet.replace(word, emojis[word])
            new_tweet.append(emojis[word])
        wordmojis = demoji.findall(word)
        for char in word:
            if char in wordmojis.keys():
                tweet = tweet.replace(word, wordmojis[char])
    
    return tweet

def remove_hashtags(tweet):
    return re.sub(r'\#w+', '', tweet)

def remove_mentions(tweet):
    for word in tweet.split():
        if word[0] == '@':
            tweet = tweet.replace(word, '')
    return tweet

def remove_punctuations(tweet):
    punct = string.punctuation
    trantab = str.maketrans(punct, len(punct)*' ')
    return tweet.translate(trantab)

def remove_url(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags = re.MULTILINE)

def clean(tweet):
    tweet = emoji2text(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_mentions(tweet)
    #tweet = remove_punctuations(tweet)
    tweet = remove_url(tweet)
    return tweet

In [ ]:
ieee['tweet'] = ieee['tweet'].astype(str)
ieee['tweet'] = ieee['tweet'].astype(str)
ieee['tweet'] = ieee['tweet'].astype(str)
ieee['tweet'] = ieee['tweet'].apply(lambda x : clean(x))

In [ ]:
ieee

In [ ]:
ieee = ieee.dropna()
ieee = ieee.drop_duplicates(['tweet'])
ieee['tweet'] = ieee['tweet'].str.lower()
ieee['month'] = ieee['month'].str.lower()
ieee['place'] = ieee['place'].str.lower()
ieee['tweet'] = ieee['tweet'].apply(lambda x : re.sub('\n', '', x))

In [ ]:
ieee = ieee.reset_index()

# **Loading Bert**

In [ ]:
bert = torch.load("/content/drive/MyDrive/Covid 19 India/bertmodel.pth")
bert

In [ ]:
bert_df = pd.DataFrame()
bert_df['Tweet'] = ieee['tweet']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * 145556
bert_df['list'] = values

In [ ]:
test_dataset = CustomDataset(bert_df, tokenizer, MAX_LEN)

In [ ]:
bert_test_params = {'batch_size': 1,
                    'shuffle': False,
                    'num_workers': 0
                    }   

test_loader = DataLoader(test_dataset, **bert_test_params)

In [ ]:
def test():
    bert.eval()
    bert_outputs = []

    with torch.no_grad():
        for unw, data in enumerate(test_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = bert(ids, mask, token_type_ids)

            bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return bert_outputs

In [ ]:
test_outputs = test()

In [ ]:
test_outputs = np.array(test_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [ ]:
bert_df['Optimistic'] = "None"
bert_df['Thankful'] = "None"
bert_df['Empathetic'] = "None"
bert_df['Pessimistic'] = "None"
bert_df['Anxious'] = "None"
bert_df['Sad'] = "None"
bert_df['Annoyed'] = "None"
bert_df['Denial'] = "None"
bert_df['Official report'] = "None"
bert_df['Surprise'] = "None"
bert_df['Joking'] = "None"
bert_df = bert_df.drop(['list'], axis = 1)

In [ ]:
for i in range(len(test_outputs)):
    bert_df['Optimistic'].iloc[i] = test_outputs[i][0]
    bert_df['Thankful'].iloc[i] = test_outputs[i][1]
    bert_df['Empathetic'].iloc[i] = test_outputs[i][2]
    bert_df['Pessimistic'].iloc[i] = test_outputs[i][3]
    bert_df['Anxious'].iloc[i] = test_outputs[i][4]
    bert_df['Sad'].iloc[i] = test_outputs[i][5]
    bert_df['Annoyed'].iloc[i] = test_outputs[i][6]
    bert_df['Denial'].iloc[i] = test_outputs[i][7]
    bert_df['Official report'].iloc[i] = test_outputs[i][8]
    bert_df['Surprise'].iloc[i] = test_outputs[i][9]
    bert_df['Joking'].iloc[i] = test_outputs[i][10]

In [ ]:
bert_df['month'] = ieee['month']
bert_df['place'] = ieee['place']

In [ ]:
bert_df.head()

In [ ]:
bert_df.to_csv("bert_final.csv", index = False)

# **Loading LSTM**

In [ ]:
model = torch.load("/content/drive/MyDrive/Covid 19 India/model7LSTM.pth")
model

In [ ]:
ieee = ieee.drop(['sentiment_score'], axis = 1)

In [ ]:
import spacy
spacy_en = spacy.load('en')

def tokenizer(tweet):
    tweet = re.sub(r'[\n]', ' ', tweet)
    return [tok.text for tok in spacy_en.tokenizer(tweet)]

tweet_field = data.Field(sequential = True, lower = True, tokenize = tokenizer)

dataFields = [("tweet", tweet_field), ("sentiment_score", None), ("place", None), ("month", None)]

train_dataset = data.TabularDataset(
    path = '/content/final.csv', format = 'csv', fields = dataFields, skip_header = True
    )

In [ ]:
print("length of dataset:", len(train_dataset))

In [ ]:
tweet_field.vocab = TWEET.vocab
BATCH_SIZE = 1
n_label = 11
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter = data.BucketIterator(train_dataset,
                                batch_size = BATCH_SIZE,
                                sort_key = lambda x : len(x.tweet),
                                sort_within_batch = False,
                                repeat = False,
                                shuffle = False,
                                device = device)

In [ ]:
train_dl = BatchWrapper(train_iter, "tweet", None)

In [ ]:
i = 0
for x, y in train_dl:
    print(x.squeeze())
    i += 1
    if i == 10: break

In [ ]:
predicted_labels = []
sus = [33484, 62825]
i = 0
for X, y in train_dl:
    i += 1
    if i not in sus:
        preds = model(X)
        sig = nn.Sigmoid()
        out = sig(preds)

        out[out >= 0.5] = 1
        out[out < 0.5] = 0
        predicted_labels.append(out.detach().cpu().numpy().tolist())
        if i % 100 == 0:
            print("{}/{} iterations done".format(i, len(train_dl)))
    else:
        predicted_labels.append([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
len(predicted_labels)

In [ ]:
till_now = predicted_labels

In [ ]:
sorted(soi['month'].value_counts())
#sep, apr, mar, jun, may, aug, jul

In [ ]:
soi['month'].value_counts()

In [ ]:
soi['Optimistic'] = "None"
soi['Thankful'] = "None"
soi['Empathetic'] = "None"
soi['Pessimistic'] = "None"
soi['Anxious'] = "None"
soi['Sad'] = "None"
soi['Annoyed'] = "None"
soi['Denial'] = "None"
soi['Official report'] = "None"
soi['Surprise'] = "None"
soi['Joking'] = "None"
soi

In [ ]:
for i in range(len(till_now)):
    soi['Optimistic'].iloc[i] = till_now[i][0]
    soi['Thankful'].iloc[i] = till_now[i][1]
    soi['Empathetic'].iloc[i] = till_now[i][2]
    soi['Pessimistic'].iloc[i] = till_now[i][3]
    soi['Anxious'].iloc[i] = till_now[i][4]
    soi['Sad'].iloc[i] = till_now[i][5]
    soi['Annoyed'].iloc[i] = till_now[i][6]
    soi['Denial'].iloc[i] = till_now[i][7]
    soi['Official report'].iloc[i] = till_now[i][8]
    soi['Surprise'].iloc[i] = till_now[i][9]
    soi['Joking'].iloc[i] = till_now[i][10]

In [ ]:
soi

In [ ]:
soi.to_csv("finaldf.csv", index = False)

In [ ]:
rand_idx = np.random.randint(0, 33483)
print(rand_idx)
print(soi.iloc[rand_idx])

In [ ]:
soi['tweet'].iloc[rand_idx]

In [ ]:
rand_idx = np.random.randint(0, 33483)
print(rand_idx)
print(soi.iloc[rand_idx])

In [ ]:
soi['tweet'].iloc[rand_idx]

In [ ]:
rand_idx = np.random.randint(0, 33483)
print(rand_idx)
print(soi.iloc[rand_idx])

In [ ]:
soi['tweet'].iloc[rand_idx]

## **Visualisations on the first 33k examples**
last edit - 28-01-2021

In [ ]:
req = soi.iloc[:33483]
req

In [ ]:
values = []
for col in req.columns[4:]:
    values.append([req[col].value_counts()[0], req[col].value_counts()[1]])

labels = req.columns[4:]
x = np.arange(len(labels))
width = 0.35

zero_means = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
one_means = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

fig, ax = plt.subplots()
plt.figure(figsize = (14, 14))
rects1 = ax.bar(x - width/2, zero_means, width, label = 'Zeros')
rects2 = ax.bar(x + width/2, one_means, width, label = 'Ones') 

ax.set_ylabel('Frequency')
#ax.set_title('')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

In [ ]:
values

In [ ]:
plt.subplots?